In [87]:
import warnings
warnings.filterwarnings("ignore")

### Load data

In [65]:
import pandas as pd
import numpy as np


data_path = 'features_annotated.csv'
df = pd.read_csv(data_path)

### Create a mapping for the URLs

In [66]:
urlSet = set()
for u in df['url'].values:
    urlSet.add(u)

url2Idx = {}
for url in urlSet:
    url2Idx[url] = len(url2Idx)

In [78]:
url2Idx

{'http://checkyourfact.com/2018/07/09/fact-check-nfl-ratings-20-percent/': 4,
 'http://www.politifact.com/personalities/sean-hannity/': 10,
 'https://africacheck.org/reports/eskom-and-the-viral-infographic-do-the-numbers-add-up/': 2,
 'https://fullfact.org/europe/uk-one-biggest-contributors-eu-budget/?utm_source=homepage&utm_medium=main_story': 8,
 'https://hoax-alert.leadstories.com/3469573-fake-news-florida-babysitter-tied-crying-one-month-old-baby-to-ceiling-fan-for-26-hours.html': 5,
 'https://pesacheck.org/have-58-of-ugandas-roads-been-built-using-only-government-money-e2156f2aa41a': 1,
 'https://theconversation.com/factcheck-is-australias-population-the-highest-growing-in-the-world-96523': 0,
 'https://theferret.scot/scotland-oil-abu-dhabi-dubai/': 6,
 'https://www.factcheck.org/2018/07/trumps-false-claims-at-nato/': 7,
 'https://www.snopes.com/fact-check/hillary-clinton-smash-phone-hammer/': 9,
 'https://www.washingtonpost.com/news/fact-checker/wp/2018/07/18/phil-bredesens-claim

### Create a mapping for the labels

In [67]:
labelSet = set()
for l in df['label'].values:
    labelSet.add(l)

label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)
labelEmbeddings = np.identity(len(label2Idx), dtype='float32')

In [68]:
label2Idx

{'Body': 0,
 'Claim': 1,
 'Credibility': 6,
 'Date': 3,
 'None': 5,
 'Sources': 4,
 'Title': 2}

### Create a mapping for the tags

In [69]:
tagSet = set()
for t in df['tag'].values:
    tagSet.add(t)

tag2Idx = {}
for tag in tagSet:
    tag2Idx[tag] = len(tag2Idx)
tagEmbeddings = np.identity(len(tag2Idx), dtype='float32')

In [79]:
tag2Idx

{'[document]': 12,
 'a': 30,
 'abbr': 3,
 'address': 42,
 'article': 63,
 'aside': 20,
 'author': 54,
 'b': 45,
 'base': 53,
 'blockquote': 47,
 'body': 55,
 'br': 16,
 'button': 24,
 'canvas': 51,
 'center': 0,
 'cite': 49,
 'detail': 59,
 'div': 8,
 'em': 7,
 'fieldset': 44,
 'figcaption': 32,
 'figure': 22,
 'footer': 31,
 'form': 65,
 'g': 27,
 'h1': 2,
 'h2': 58,
 'h3': 66,
 'h4': 71,
 'h5': 62,
 'h6': 50,
 'head': 57,
 'header': 9,
 'hr': 35,
 'html': 60,
 'i': 21,
 'iframe': 41,
 'img': 56,
 'input': 15,
 'ins': 64,
 'label': 5,
 'legend': 10,
 'li': 19,
 'link': 29,
 'main': 39,
 'meta': 18,
 'nav': 25,
 'noscript': 4,
 'ol': 17,
 'option': 52,
 'p': 37,
 'path': 46,
 'photo': 38,
 'section': 11,
 'select': 69,
 'sidebar': 26,
 'small': 61,
 'snippet': 48,
 'span': 1,
 'strong': 23,
 'style': 43,
 'sup': 14,
 'svg': 33,
 'symbol': 13,
 'text': 28,
 'textarea': 36,
 'time': 67,
 'title': 40,
 'u': 34,
 'ul': 70,
 'use': 68,
 'wp-ad': 6}

##### ToDo - Improve level feature representation

In [70]:
levelEmbeddings = np.identity(max(df['level'].values), dtype='float32')

### Data to features

In [71]:
columns = ['url', 'label', 'tag', 'level']  # 0 url  1 label

features = {'tags': [], 'levels': [], 'labels': [], 'groups': []}

In [72]:
for data in df[columns].values:
    features['groups'].append(url2Idx[data[0]])
    features['labels'].append(labelEmbeddings[label2Idx[data[1]]])
    features['tags'].append(tagEmbeddings[tag2Idx[data[2]]])
    features['levels'].append(levelEmbeddings[data[3]-1])
features['tags'] = np.array(features['tags'])
features['levels'] = np.array(features['levels'])
features['labels'] = np.array(features['labels'])
features['groups'] = np.array(features['groups'])

### Util - Plot confusion matrix

In [73]:
import matplotlib.pyplot as plt
import itertools


def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Dense Model

In [74]:
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate


def dense_model():
    tag_input = Input(shape=(tagEmbeddings.shape[0],))
    tag_ = Dense(32, activation='relu')(tag_input)
    tag_ = Dense(16, activation='relu')(tag_)

    level_input = Input(shape=(levelEmbeddings.shape[0],))
    level_ = Dense(32, activation='relu')(level_input)
    level_ = Dense(16, activation='relu')(level_)

    output = concatenate([tag_, level_])
    output = Dense(32, activation='relu')(output)
    output = Dense(16, activation='relu')(output)
    output = Dense(len(label2Idx), activation='softmax')(output)

    model = Model(inputs=[tag_input, level_input], outputs=[output])
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

#### Cross Validation - Dense Model

In [ ]:
from sklearn.model_selection import GroupKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


accuracy = []
precision = []
recall = []
f1 = []
confusion = []

gp = GroupKFold(n_splits=8)
for train_indexs, test_indexs in gp.split(features['tags'], groups=features['groups']):
    
    tags_train = features['tags'][train_indexs]
    levels_train = features['levels'][train_indexs]
    X_train = [tags_train, levels_train]
    y_train = features['labels'][train_indexs]
    
    tags_test = features['tags'][test_indexs]
    levels_test = features['levels'][test_indexs]
    X_test = [tags_test, levels_test]
    y_test = features['labels'][test_indexs]

    groups_train = features['groups'][train_indexs]
    groups_test = features['groups'][test_indexs]
    train_g = set()
    test_g = set()
    for g in groups_train:
        train_g.add(g)
    for g in groups_test:
        test_g.add(g)
    print('Train Groups (URLs)', train_g)
    print('Test Groups (URLs)', test_g)
    
    model = KerasClassifier(build_fn=dense_model)
    model.fit(X_train, y_train, epochs=5, verbose=0)

    result = model.predict(X_test)
    y_test = [r.tolist().index(1) for r in y_test]
    acc = accuracy_score(result, y_test)
    accuracy.append(acc)
    p = precision_score(result, y_test, average="macro")
    precision.append(p)
    r = recall_score(result, y_test, average="macro")
    recall.append(r)
    f = f1_score(result, y_test, average="macro")
    f1.append(f)
    confusion.append(confusion_matrix(result, y_test))
    
    print("%s: %.2f %%" % ('Acc', acc*100))
    print("%s: %.2f %%" % ('Precision', p*100))
    print("%s: %.2f %%" % ('Recall', r*100))
    print("%s: %.2f %%" % ('F1', f*100))
    print('')

print("Acc %.2f %% (+/- %.2f %%)" % (np.mean(accuracy)*100, np.std(accuracy)*100))
print("Precision %.2f %% (+/- %.2f %%)" % (np.mean(precision)*100, np.std(precision)*100))
print("Recall %.2f %% (+/- %.2f %%)" % (np.mean(recall)*100, np.std(recall)*100))
print("F1 %.2f %% (+/- %.2f %%)" % (np.mean(f1)*100, np.std(f1)*100))

Train Groups (URLs) {0, 1, 3, 4, 5, 6, 7, 8, 9, 10}
Test Groups (URLs) {2}
Acc: 90.38 %
Precision: 25.50 %
Recall: 23.07 %
F1: 24.20 %

Train Groups (URLs) {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Test Groups (URLs) {0}
Acc: 90.33 %
Precision: 28.28 %
Recall: 27.43 %
F1: 27.85 %

Train Groups (URLs) {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Test Groups (URLs) {10}
Acc: 96.19 %
Precision: 14.24 %
Recall: 13.78 %
F1: 14.01 %

Train Groups (URLs) {0, 1, 2, 3, 4, 5, 6, 8, 9, 10}
Test Groups (URLs) {7}
Acc: 86.29 %
Precision: 14.29 %
Recall: 12.33 %
F1: 13.23 %

Train Groups (URLs) {0, 1, 2, 3, 4, 5, 6, 7, 8, 10}
Test Groups (URLs) {9}


###### Confusion Matrix plot

In [ ]:
nc = []
for c in confusion:
    d = len(label2Idx) - len(c)
    c = np.pad(c, (0,d), 'constant')
    nc.append(c)
confusion = nc
cm = np.mean(confusion, axis=0)
plot_confusion_matrix(cm, ['Body', 'Claim', 'Title', 'Date', 'Sources', 'None', 'Credibility'], normalize=True)

### LSTM model

In [ ]:
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate


tag_input = Input(shape=(None,), dtype='int32', name='tags_input')
tag = Embedding(output_dim=tagEmbeddings.shape[1], input_dim=tagEmbeddings.shape[0], weights=[tagEmbeddings], trainable=False)(tag_input)

level_input = Input(shape=(None,), dtype='int32', name='levels_input')
level = Embedding(output_dim=levelEmbeddings.shape[1], input_dim=levelEmbeddings.shape[0], weights=[levelEmbeddings], trainable=False)(level_input)

output = concatenate([tag, level])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)

model = Model(inputs=[tag_input, level_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
model.summary()
#model.fit([tag_features, level_features], labels, epochs=10)